# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import t
from scipy.stats import norm
from scipy.stats import ttest_1samp
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

%matplotlib inline

In [2]:
!pip install statsmodels

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [3]:
# Your code here:
df=pd.read_csv('/home/carmencuadrado/Ironhack/dataptmad0420/module-2/hypothesis-testing/hypothesis-testing.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:
df.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
df.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
# Your code here:
df.groupby(['Salary or Hourly']).count()['Name']

Salary or Hourly
Hourly     8022
Salary    25161
Name: Name, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [7]:
# Your code here:
df.groupby(['Department']).count()['Name']

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [8]:
# Your code here:
data=df[df['Salary or Hourly']=='Hourly']['Hourly Rate']
sample_size=len(data)
mean_hwage=np.mean(data)
print('The mean is', mean_hwage,'therefore it is different from 30. We can go on conducting the hypothesis testing')

The mean is 32.78855771628024 therefore it is different from 30. We can go on conducting the hypothesis testing


In [9]:
std_hwage=np.std(data)
t_test = (mean_hwage - 30) / (std_hwage / np.sqrt(sample_size))
print('T equals',t_test, 'which makes us think that the hourly wage of all hourly workers is significantly different from 30')

T equals 20.621091109215907 which makes us think that the hourly wage of all hourly workers is significantly different from 30


In [10]:
test_results = ttest_1samp(data, 30)
p_value=test_results.pvalue
print('However, the p_value equals',p_value,'which leads us to conclude that the null hypothesis should be rejected as the p-value is <0.05%. In conclusion, the hourly wage of all hourly workers is significantly different from $30/hr. ')

However, the p_value equals 4.3230240486229894e-92 which leads us to conclude that the null hypothesis should be rejected as the p-value is <0.05%. In conclusion, the hourly wage of all hourly workers is significantly different from $30/hr. 


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [11]:
# Your code here:
data=df[df['Salary or Hourly']=='Salary']['Annual Salary']
sample_size=len(data)
mean_hwage=np.mean(data)
print('We can go on with the test as the sample mean is:', mean_hwage, 'Thus, greater than $86000')

We can go on with the test as the sample mean is: 86786.99979015143 Thus, greater than $86000


In [12]:
std_hwage=np.std(data)
t_test = (mean_hwage - 86000) / (std_hwage / np.sqrt(sample_size))
t_test

5.932988417351495

In [13]:
test_results = ttest_1samp(data, 86000)
p_value=(test_results.pvalue /2)
print('We finnaly conclude that the null hypothesis should be rejected: Salaries are higher than last year as the p_value is:',p_value)

We finnaly conclude that the null hypothesis should be rejected: Salaries are higher than last year as the p_value is: 1.507879259958193e-09


Using the `crosstab` function, find the department that has the most hourly workers. 

In [14]:
# Your code here:
pd.crosstab(index=df['Department'],columns=df[df['Salary or Hourly']=='Salary']['Salary or Hourly'], values=df['Salary or Hourly'],aggfunc='count').idxmax()

Salary or Hourly
Salary    POLICE
dtype: object

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [15]:
# Your code here:
x=df['Salary or Hourly']=='Hourly'
y=df['Department']=='POLICE'
data=df[x&y]['Hourly Rate']
sample_size=len(data)
mean_phwage=np.mean(data)
print('We can go on with the test as the sample mean is:', mean_phwage, 'Thus, smaller than $35/hour')

We can go on with the test as the sample mean is: 9.384 Thus, smaller than $35/hour


In [16]:
std_phwage=np.std(data)
t_test = (mean_phwage - 35) / (std_phwage / np.sqrt(sample_size))
print('t is', t_test,'which is a negative value taking us believe that their salary is less than $35/hour')

t is -870.2661526004257 which is a negative value taking us believe that their salary is less than $35/hour


In [17]:
test_results = ttest_1samp(data, 35)
p_value=(test_results.pvalue /2)
print('This result leads us to unequivocally state that policy hourly wage is less than $35/hour, as the p-value is:', p_value)

This result leads us to unequivocally state that policy hourly wage is less than $35/hour, as the p-value is: 1.4284243665683163e-23


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [18]:
# Your code here:
data=df[df['Salary or Hourly']=='Hourly']['Hourly Rate']
print('The confidence interval equals,',(t.interval(0.95, data))[1])
print('while the standard error is',stats.sem(data, ddof=1))

The confidence interval equals, [2.08690653 2.01277777 2.02888554 ... 2.02888554 2.0124855  2.00973133]
while the standard error is 0.1352368565101596


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [19]:
# Your code here:
x=df['Salary or Hourly']=='Salary'
y=df['Department']=='POLICE'
data=df[x&y]['Annual Salary']
print('The confidence interval equals,',(t.interval(0.95, data))[1])
print('while the standard error is',stats.sem(data, ddof=1))

The confidence interval equals, [1.95998737 1.95998919 1.9599894  ... 1.96001333 1.95999034 1.9599894 ]
while the standard error is 157.82445675052244


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [26]:
# Your code here:
data=df[df['Salary or Hourly']=='Hourly']

count = len(data)
nobs = len(df)
value = .25
stat, pval = proportions_ztest(count, nobs, value, alternative='two-sided')
print('We can reject the null hypothesis as the p-value is','{0:0.2}'.format(pval),'.Therefore, there is no doubt that the proportion of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.') 

We can reject the null hypothesis as the p-value is 0.00045 .Therefore, there is no doubt that the proportion of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.
